In [14]:
import os
from selenium import webdriver 
from time import sleep
from selenium.webdriver.support.ui import Select
import MySQLdb
import ast
import datetime

In [15]:
def send_keys(idname, key, clear=True):
    element = driver.find_element_by_id(idname)
    
    if clear:
        element.clear()
    element.send_keys(key)
    
def select_text(idname, text):
    Select(driver.find_element_by_id(idname)).select_by_visible_text(text);

def select_idx(idname, idx):
    Select(driver.find_element_by_id(idname)).select_by_index(idx);
    
def click(idname):
    if type(idname) is str:
        driver.find_element_by_id(idname).click()
    else :
        idname.click()
        
def click_by_css(css):
    driver.find_element_by_css_selector(css).click();

In [16]:

def connect():
    db = MySQLdb.connect(
        "127.0.0.1",
        "root",
        "fbendud89",
        "soulfood",
        charset='utf8',) 
    return db 
    
def select_combine():
    SQL_QUERY = "select * from `combine`"

    curs = db.cursor()
    count = curs.execute(SQL_QUERY)
    rows = curs.fetchall()
    return rows

def select_item(id):
    SQL_QUERY = "select * from `item` where id = {}".format(id)

    curs = db.cursor()
    count = curs.execute(SQL_QUERY)
    rows = curs.fetchall()
    return rows

In [17]:
db = connect()
combis = select_combine()

In [18]:
def step_1(title, i_type):
    driver.get("http://www.esmplus.com/Home/")

    click(driver.find_elements_by_id('rdoSiteSelect')[1])
    send_keys("SiteId", "cnhfood12")
    send_keys("SitePassword", "thdnf12#45") 
    click("btnSiteLogOn")
    driver.execute_script("addTabReload('TDM001')")

    iframes = None
    while True:
        try: 
            click(driver.find_element_by_css_selector("#TDM352 > a"))
            iframes = driver.find_elements_by_id("ifm_TDM352"); 
            if len(iframes) > 0:
                iframe = iframes[0] 
                break
            else:
                sleep(0.2)
        except Exception as e:
            print(e)
            sleep(0.2)

    driver.switch_to_frame(iframe);

    select_text("stSiteLCategory2","가공식품");
    if i_type.startswith("캔디"):
        select_text("stSiteMCategory2","캔디/껌/젤리");
        select_text("stSiteSCategory2","사탕/캔디"); 
    else:
        select_text("stSiteMCategory2","과자/안주");
        select_text("stSiteSCategory2","비스킷/크래커"); 
        
    send_keys("txtGoodsName", title)

    #driver.find_element_by_id("show_next_step").click()
    driver.find_element_by_css_selector("#show_next_step > img").click()
     
    return iframes[0]

In [19]:
def step_2(options, iframe, c_price):
    period = None
    while True:
        periods = driver.find_elements_by_id("sltFixedPriceSellingPeriod");
        if len(periods) > 0:
            period = periods[0]
            break;
        else:
            sleep(0.2)

    select_text("sltFixedPriceSellingPeriod", "90일")


    send_keys("txtGoodsPrice", c_price)
    send_keys("txtGoodsCount", 99999)

    parent = driver.current_window_handle
    optioncnt = 2

    if optioncnt > 1:
        click("rdoGoodsOptionsUseSettingTrue")
        click("btnOptionsPopup")

        driver.switch_to_window(driver.window_handles[1])
    
    click_by_css("#popContents > div:nth-child(1) > div.btn_center > a > img")
    sleep(0.2)
    click_by_css("#step2_first_table > table > tbody > tr > td.first-child > div > div:nth-child(2) > div > span > a:nth-child(1)")
    sleep(0.2)
    #click_by_css("#step2_first_table > table > tbody > tr > td.first-child > div > div:nth-child(2) > div > div > div > div > ul > li:nth-child(1) > a")
    click_by_css("#step2_first_table > table > tbody > tr > td.first-child > div > div:nth-child(2) > div > div > p > a")

    input_option_name = driver.find_element_by_css_selector('#step2_first_table > table > tbody > tr > td.first-child > div > div:nth-child(1) > input')
    input_option_name.clear()
    input_option_name.send_keys('선택')

    input_option = driver.find_element_by_css_selector('#step2_first_table > table > tbody > tr > td:nth-child(2) > div > div:nth-child(1) > input')
    input_option.clear()
    input_option.send_keys(','.join(options))

    click_by_css("#popContents > div:nth-child(2) > div.btn_center > a:nth-child(2) > img")

    click_by_css("#save > img")

    driver.switch_to_window(parent)
    driver.switch_to_frame(iframe)

In [24]:
def step_3(img_ths, imgs1, imgs2):
    

    if img_ths[0]:
        input_thumbnail = driver.find_element_by_css_selector("#divImgUploader > div.s_img_registration > ul > li.ibox.fir > div > form:nth-child(1) > span.reg_btn_full > input")
        input_thumbnail.send_keys(img_ths[0]);
    
    if len(img_ths)>1:
        if img_ths[1]:
            input_thumbnail = driver.find_element_by_css_selector("#divImgUploader > div.s_img_registration > ul > li:nth-child(2) > div > form:nth-child(1) > span.reg_btn_full > input")
            input_thumbnail.send_keys(img_ths[1]);

        if img_ths[2]:
            input_thumbnail = driver.find_element_by_css_selector("#divImgUploader > div.s_img_registration > ul > li:nth-child(3) > div > form:nth-child(1) > span.reg_btn_full > input")
            input_thumbnail.send_keys(img_ths[2]);
        
        
    for img in imgs1: 
        imgurl = "<img src='http://gi.esmplus.com/cnhfood12/work/{0}'/>".format(img)
        send_keys("txtAreaEditorNew", imgurl, False)
        
    for img in imgs2:
        imgurl = "<img src='http://gi.esmplus.com/cnhfood12/work/{0}'/>".format(img)
        send_keys("txtAreaEditorNew", imgurl, False)
        
    
    imgurl = "<img src='http://gi.esmplus.com/cnhfood12/work/event.jpg'/>"
    send_keys("txtAreaEditorNew", imgurl, False)

In [25]:
def step_4():
    send_keys("txtManufacturedDateYear", datetime.datetime.now().year)
    send_keys("txtManufacturedDateMonth", datetime.datetime.now().month)
    send_keys("txtManufacturedDateDay", datetime.datetime.now().day)
    click("rdoOriginProductTypeInside")

    select_idx("transTypeChoiceGmkt", 1)
    sleep(0.2)
    click("rdoGmktDeliveryWayOPTSEL1")
    select_text("selGmktDeliveryWayOPTSELParcelCOMP", "대한통운")
    select_idx("selShipmentPlaceNo",1)
    sleep(0.2)
    select_idx("selBundleDeliveryTemp",1)
    sleep(0.2)
    select_idx("ddlOfficialNotiGroup",21)
    sleep(0.5)
    txts = driver.find_elements_by_css_selector(".official-notice_default textarea._resizeHeight")
     
    
    txts[0].send_keys("과자(유처리제품)");
    txts[1].send_keys("식품위생법에 따른수입 신고를 필함");
    txts[2].send_keys("070-4151-2465");
    txts[3].send_keys("3일이내");
    txts[4].send_keys("상세설명참조");
    txts[5].send_keys("제품별도표기");
    txts[6].send_keys("제품별도표기");
    txts[7].send_keys("상세설명참조");
    txts[8].send_keys("상세설명참조");
    txts[9].send_keys("상세설명참조");
    txts[10].send_keys("상세설명참조");
    txts[11].send_keys("상세설명참조");
    click_by_css("#floating_reg_btn > a:nth-child(1) > img")
    
    
    parent = driver.current_window_handle
    driver.switch_to_window(driver.window_handles[1])
    sleep(0.5)
        
    click("lblConfirmForGoodsName")
    sleep(0.1)
    click("lbConfirmForGoodsImage")
    sleep(0.1)
    click("lblConfirmForGoodsPrice")
    sleep(0.1)
    click_by_css("#btnConfirm > img")

    sleep(3)
    driver.switch_to_window(driver.window_handles[0])

    iframe = driver.find_element_by_id("ifm_TDM352")
    driver.switch_to_frame(iframe)
    sleep(0.2)
    addid = driver.find_element_by_css_selector("#contents > div.basic_contents > div.product_complete > div.product_regist > div > table > tbody > tr:nth-child(1) > td > span")
    return addid.text

In [26]:
result = []
def add_gmarket(combi):
    c_id = combi[0]
    c_type = combi[1]
    c_title = combi[2]
    c_price = combi[3]
    c_data = ast.literal_eval(combi[4])
    
    files = !ls /Users/duyeoungryu/Documents/dev/flask/soulfood/static/img/work
    img_ths = []
    for fn in files:
        if fn.startswith("th_{}_".format(c_id)):
            img_ths.append("/Users/duyeoungryu/Documents/dev/flask/soulfood/static/img/work/" + fn)
            
    img_mains = []
    img_nutrs = []
    if c_type == 0:
        keys = list(c_data.keys())
        karr = keys[len(keys)-1].split(',')
    else:
        karr = list(c_data.keys()) 
        

    i_type = []
    for iid in karr:
        item = select_item(iid) 
        i_type.append(item[0][5])

        img_m = "{}_main.jpg".format(iid)
        img_n = "{}_nutr.jpg".format(iid)
        if img_m in files: 
            img_mains.append(img_m)
        else:
            print('no ', iid)

        if img_n in files: 
            img_nutrs.append(img_n)
        else:
            print('no ', iid)
    
    
    options = []
    for k, v in c_data.items(): 
        karr = k.split(',')
        name = ''
        if len(karr) > 1: 
            names = []
            for s_k in karr:
                item = select_item(s_k)
                name = item[0][1]
                names.append(name)
            name = (' + '.join(names))
        else:
            item = select_item(k)
            name = item[0][1]
            name = "{} x {}".format(name, v)
    
        options.append(name)
     
    i_type = list(set(i_type))  
    iframe = step_1(c_title, i_type[0]) 
    step_2(options, iframe, c_price)
    step_3(img_ths, img_mains, img_nutrs)
    addid = step_4()
    
    result.append([c_id, addid])

In [41]:
driver = webdriver.Chrome()
add_gmarket(combis[23])
driver.close()

In [38]:
for idx in range(11, 24): 
    driver = webdriver.Chrome()
    add_gmarket(combis[idx])
    driver.close()
    sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"id","selector":"lblConfirmForGoodsName"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


In [42]:
result

[[4, '1473941994'],
 [5, '1473942287'],
 [6, '1473942490'],
 [7, '1473942933'],
 [8, '1473943515'],
 [9, '1473943744'],
 [10, '1473944018'],
 [12, '1473945677'],
 [13, '1473945977'],
 [14, '1473946263'],
 [15, '1473946539'],
 [16, '1473946733'],
 [17, '1473946937'],
 [18, '1473947134'],
 [19, '1473947314'],
 [20, '1473947487'],
 [21, '1473947711'],
 [22, '1473948002'],
 [23, '1473948200'],
 [24, '1473950315']]